<a href="https://colab.research.google.com/github/zachfreitas/ADS_505_Applied_Data_Sci_for_Business-/blob/main/Module_3/Assignment_3/Zachariah_Freitas_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3.1
Zachariah Freitas <br>
2022-09-24

Import required packages

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install dmba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 6.6 MB/s 


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

import matplotlib.pylab as plt

from dmba import plotDecisionTree, gainsChart, liftChart
from dmba import classificationSummary, regressionSummary
import dmba

%matplotlib inline

no display found. Using non-interactive Agg backend


## Data ETL and EDA

In [6]:
# Load the data
taxi_df = pd.read_csv('https://raw.githubusercontent.com/zachfreitas/ADS_505_Applied_Data_Sci_for_Business-/main/Module_3/Assignment_3/Taxi-cancellation-case.csv')
taxi_df.head()

,row#,user_id,vehicle_model_id,package_id,travel_type_id,from_area_id,to_area_id,from_city_id,to_city_id,from_date,to_date,online_booking,mobile_site_booking,booking_created,from_lat,from_long,to_lat,to_long,Car_Cancellation
0,1,17712,12,NaN,2,1021.0,1323.0,NaN,NaN,1/1/13 22:33,NaN,0,0,1/1/13 8:01,13.028530,77.54625,12.869805,77.653211,0
1,2,17037,12,NaN,2,455.0,1330.0,NaN,NaN,1/1/13 12:43,NaN,0,0,1/1/13 9:59,12.999874,77.67812,12.953434,77.706510,0
2,3,761,12,NaN,2,814.0,393.0,NaN,NaN,1/2/13 0:28,1/3/13 0:00,1,0,1/1/13 12:14,12.908993,77.68890,13.199560,77.706880,0
3,4,868,12,NaN,2,297.0,212.0,NaN,NaN,1/1/13 13:12,NaN,0,0,1/1/13 12:42,12.997890,77.61488,12.994740,77.607970,0
4,5,21716,28,NaN,2,1237.0,330.0,NaN,NaN,1/1/13 16:33,NaN,0,0,1/1/13 15:07,12.926450,77.61206,12.858833,77.589127,0


In [7]:
taxi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   row#                 10000 non-null  int64  
 1   user_id              10000 non-null  int64  
 2   vehicle_model_id     10000 non-null  int64  
 3   package_id           1752 non-null   float64
 4   travel_type_id       10000 non-null  int64  
 5   from_area_id         9985 non-null   float64
 6   to_area_id           7909 non-null   float64
 7   from_city_id         3706 non-null   float64
 8   to_city_id           339 non-null    float64
 9   from_date            10000 non-null  object 
 10  to_date              5822 non-null   object 
 11  online_booking       10000 non-null  int64  
 12  mobile_site_booking  10000 non-null  int64  
 13  booking_created      10000 non-null  object 
 14  from_lat             9985 non-null   float64
 15  from_long            9985 non-null   

In [9]:
taxi_df.describe().T

,count,mean,std,min,25%,50%,75%,max
row#,10000.0,5000.500000,2886.895680,1.00000,2500.750000,5000.500000,7500.25000,10000.000000
user_id,10000.0,30663.966200,10988.599834,16.00000,24410.750000,31510.000000,39094.50000,48729.000000
vehicle_model_id,10000.0,26.191700,27.228053,1.00000,12.000000,12.000000,24.00000,91.000000
package_id,1752.0,1.988014,1.401791,1.00000,1.000000,2.000000,2.00000,7.000000
travel_type_id,10000.0,2.141300,0.434917,1.00000,2.000000,2.000000,2.00000,3.000000
from_area_id,9985.0,709.767752,418.968918,2.00000,393.000000,590.000000,1086.00000,1401.000000
to_area_id,7909.0,665.457833,400.628562,6.00000,393.000000,516.000000,1052.00000,1403.000000
from_city_id,3706.0,14.898003,1.190770,1.00000,15.000000,15.000000,15.00000,15.000000
to_city_id,339.0,68.318584,49.886427,4.00000,32.000000,49.000000,108.00000,203.000000
online_booking,10000.0,0.353300,0.478019,0.00000,0.000000,0.000000,1.00000,1.000000


---
## Question 1
How can a predictive model based on these data be used by Yourcabs.com? (10
Points)


---
**Question 1 Answer:**
Ugh
---

---
## Question 2
How can a profiling model (identifying predictors that distinguish canceled/uncanceled trips) be used by Yourcabs.com? (10 Points)

---
**Question 2 Answer:**
Ugh
---

---
## Question 3
Explore, prepare, and transform the data to facilitate predictive modeling. Here are some
hints: (15 Points)

---
### Question 3.1
Explore, prepare, and transform the data to facilitate predictive modeling. Here are some
hints: (15 Points)

---
**Question 3.1 Answer:**<br>
ugh
---

---
### Question 3.2
How will you deal with missing data, such as cases where NaN is indicated?



---
**Question 3.2 Answer:**<br>
ugh
---

---
### Question 3.3
Think about what useful information might be held within the date and time fields (the booking timestamp and the trip timestamp).


---
**Question 3.3 Answer:**<br>
ugh
---

---
### Question 3.4
Think also about the categorical variables, and how to deal with them. Should we turn them all into dummies? Use only some?

---
**Question 3.4 Answer:**<br>
ugh
---

---
## Question 4
Fit several predictive models of your choice. Do they provide information on how the predictor variables relate to cancellations? (15 Points)


---
**Question 4 Answer:**<br>
ugh
---

---
## Question 5
Report the predictive performance of your model in terms of error rates (the confusion matrix). How well does the model perform? Can the model be used in practice? (10 Points)

---
**Question 5 Answer:**<br>
ugh
---

---
## Question 6
Examine the predictive performance of your model in terms of ranking (lift). How well does the model perform? Can the model be used in practice? (10 Points)


---
**Question 6 Answer:**<br>
ugh
---

---
## Question 7
Briefly explain, in two to three paragraphs, the business objective, the data mining models used, why they were used, the model results, and your recommendations to your non-technical stakeholder team. (10 Points)


---
**Question 7 Answer:**<br>
ugh
---